In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
1,26,SI,VIGENTE,TRÁNSITO RESTRINGIDO,11/12/2022,22/01/2023 21:00 HORAS,APURIMAC/ABANCAY/PICHIRHUA,KM 415+500,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-13.89561,...,-13.89561+-73.04456,2023 21:00 HORAS+01+22+21:00,0,0,CONCESIONARIA SURVIAL,HUMANO,PE-30A,CARRETERA NASCA - PUQUIO - ABANCAY,SI,restringido
2,54,SI,VIGENTE,TRÁNSITO RESTRINGIDO,11/01/2023,22/01/2023 21:00 HORAS,APURIMAC/COTABAMBAS/TAMBOBAMBA,KM 255,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-13.94967171,...,-13.94967171+-72.1797879,2023 21:00 HORAS+01+22+21:00,0,0,COE-MTC,HUMANO,PE-3SF,CARRETERA ABANCAY - CHALHUAHUACHO - CHINCHAYPUJIO,SI,restringido
4,218,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,10/01/2023,22/01/2023 21:00 HORAS,MADRE DE DIOS/TAHUAMANU/IBERIA,KM 597,(SECTOR IBERIA) PARO INDEFINIDO SOLICITANDO NU...,-11.4068843,...,-11.4068843+-69.4784365,2023 21:00 HORAS+01+22+21:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
5,219,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,14/01/2023,22/01/2023 21:00 HORAS,MADRE DE DIOS/TAHUAMANU/TAHUAMANU,KM 549,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-11.665167,...,-11.665167+-69.229412,2023 21:00 HORAS+01+22+21:00,0,0,IIRSA SUR / UD MADRE DE DIOS,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
6,229,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,06/01/2023,22/01/2023 21:00 HORAS,MADRE DE DIOS/TAMBOPATA/LAS PIEDRAS,KM 511+800,(PUENTE MAVILA) PARO INDEFINIDO SOLICITANDO NU...,-11.931332,...,-11.931332+-69.11748,2023 21:00 HORAS+01+22+21:00,0,0,IIRSA SUR,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,263,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,07/01/2023,22/01/2023 21:00 HORAS,PUNO/EL COLLAO/CAPAZO,KM 171,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-17.19684952581,...,-17.19684952581+-69.759059671272,2023 21:00 HORAS+01+22+21:00,0,0,COE-MTC,HUMANO,PE-36A,CARRETERA TACNA - TARATA - MAZOCRUZ,SI,interrumpido
81,334,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,11/12/2022,22/01/2023 21:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 390,SECTOR HUIPOCA. PARO INDEFINIDO SOLICITANDO NU...,-8.976077,...,-8.976077+-75.44483,2023 21:00 HORAS+01+22+21:00,0,0,CEOPOL,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
82,335,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,19/01/2023,22/01/2023 21:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 391,GRUPO DE PERSONAS PROTESTAN SOLICITANDO NUEVA...,-8.9903469087213,...,-8.9903469087213+-75.452262878851,2023 21:00 HORAS+01+22+21:00,0,0,UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
83,170,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,22/01/2023,22/01/2023 21:00 HORAS,HUANUCO/HUANUCO/CHURUBAMBA,KM 25,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-9.831249,...,-9.831249+-76.096243,2023 21:00 HORAS+01+22+21:00,0,0,CEOPOL,HUMANO,PE-18A,CARRETERA HUANUCO - TINGO MARIA,SI,interrumpido
